In [2]:
from photutils.aperture import CircularAperture, EllipticalAperture, aperture_photometry
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.io import fits
import numpy as np
from astropy.cosmology import Planck15

def image_reader(fits_file_path):
    """
    Read a fits file and return data and header separately.
    If data contains axes of length 1, remove these axes.
    """    
    im = fits.open(fits_file_path)
    header = im[0].header
    array2d = np.squeeze(im[0].data) #remove axes of length 1
    return(array2d, header)

def aper_pos(header, ra=None, dec=None):
    """
    Use header to get the WCS.
    Convert given aperture RA and Dec position (in SkyCoord accepted format) into pixel x,y position.
    Return x, y in pixels, and pixel size in degrees.
    (assuming this information is present in the header).
    """
    w = WCS(header,naxis=2)
    center = SkyCoord(ra, dec, unit=(u.hourangle, u.deg),frame='fk5')
    x,y = w.world_to_pixel(center)
    pix_len = header['CDELT2']
    return(x,y,pix_len)

def beam_area_pix(header):
    """
    Get beam properties and pixel size from header.
    Return beam size in pixels.
    """    
    pix_len = header['CDELT2']
    beam_maj = header['BMAJ']
    beam_min = header['BMIN']
    beam_area_deg = (np.pi / (4*np.log(2.))) * beam_maj*beam_min
    npix_per_beam = beam_area_deg/pix_len**2
    return(npix_per_beam)
    
def flux_calculator(array2d, header, ra=None, dec=None, aper=None):
    """
    Calculate flux (in Jy or Jy-km/s if moment-0 map)
    within given aperture centered at given RA and Dec coordinates.
    If no RA and Dec are given, use the center of the given 2D array.
    If no aperture is given, use a 2.5"x1.5" elliptical aperture oriented at -30 degrees.
    """
    if (ra == None) & (dec == None):
        npixx, npixy = array2d.shape
        galx = npixx/2
        galy = npixy/2
        pix_len = header['CDELT2']
    else:
        galx,galy,pix_len = aper_pos(header, ra=ra, dec=dec)
    
    if aper is None:
        aper = EllipticalAperture((galx,galy), a = 2.5/(pix_len*3600), b = 1.5/(pix_len*3600), theta = -np.pi/6) #arcsec to pixel conversion
        
    tab = aperture_photometry(array2d, apertures=aper)
    
    flux_per_beam = tab['aperture_sum'][0]
    npix_per_beam = beam_area_pix(header)
    flux_tot = flux_per_beam/npix_per_beam
    
    return(flux_tot) #Jy not Jy/beam

def luminosity(array2d, header, ra=None, dec=None, nu_obs=233, z=7, aper=None):
    """
    Calculate luminosity from flux using Eq.18 from Casey et al 2014.
    
    Observed frequency (nu_obs) and redshift (z) must be provided;
    defaults are 233GHz and 7 respectively.
    
    Return luminosity in multiples of solar luminosity.
    """
    D_L = Planck15.luminosity_distance(z).value #calculate luminosity distance from redshift
    S_nu_kmps = flux_calculator(array2d, header, ra=ra, dec=dec, aper=aper)
    L = 1.04e-3 * S_nu_kmps * nu_obs * (D_L**2) # L_sun
    return(L)

def find_ratio(arr1, hed1, arr2, hed2, ra=None, dec=None, nu_obs1=233, nu_obs2=233, flux_or_lum='flux', aper=None):
    """
    Calculate ratio of fluxes or luminosities (specify using flux_or_lum parameter)
    between two maps at given RA and Dec position.
    
    The observed frequencies of each line (nu_ob1 and nu_obs2)
    must be provided to calculate luminosity ratio.
    
    If no RA and Dec is given for the aperture,
    calculated fluxes are at the center pixel of the respective maps,
    which may be at different physical positions on the sky.
    So, resulting flux ratio may not be physically informative or useful.
    """
    f1 = flux_calculator(arr1, hed1, ra=ra, dec=dec, aper=aper)
    f2 = flux_calculator(arr2, hed2, ra=ra, dec=dec, aper=aper)
    ratio = f1/f2
    if flux_or_lum == 'lum':
        ratio = ratio*(nu_obs1/nu_obs2)
    return(ratio)